In [1]:
import pandas as pd
import numpy as np
from scipy import stats
from pandas_visual_analysis import VisualAnalysis
import matplotlib.pyplot as plt
import json
from collections import OrderedDict
from datetime import date
import time
import seaborn as sb

In [2]:
data = pd.read_csv('./csv/abusingDetectionTrainDataset.csv')
data.shape

(104399, 44)

In [3]:
data = data.drop(['socialBooleanB', 'accountMetaAmountA', 'charSessionAmountB', 'tradeSessionAmountC', 'tradeSessionAmountE'], axis=1)
data.shape

(104399, 39)

In [4]:
data.columns

Index(['newID', 'char_jobcode', 'char_level', 'logging_timestamp', 'charStatA',
       'charStatB', 'charStatC', 'charStatD', 'charStatE', 'charStatF',
       'charStatG', 'socialAmountA', 'socialBooleanA', 'socialSessionAmountA',
       'activityCumulativeAmountA', 'charSessionAmountA',
       'charMoveSessionAmountA', 'charSessionAmountC', 'socialSessionAmountB',
       'actionSessionAmountA', 'charSessionAmountD',
       'activityCumulativeAmountB', 'actionSessionAmountB',
       'actionSessionAmountC', 'actionSessionAmountD', 'actionSessionAmountE',
       'tradeSessionAmountA', 'tradeSessionAmountB', 'actionSessionAmountF',
       'actionSessionAmountG', 'actionSessionAmountH', 'actionSessionAmountI',
       'actionSessionAmountJ', 'actionSessionAmountK', 'actionSessionAmountL',
       'actionSessionAmountM', 'actionSessionAmountN', 'tradeSessionAmountD',
       'blocked'],
      dtype='object')

# char_level -> one-hot encoding

In [11]:
min(data['char_jobcode'])

0

In [13]:
max(data['char_jobcode'])

165

In [15]:
level_onehot = pd.get_dummies(data['char_jobcode'])

In [19]:
level_onehot

,0,1,2,3,4,5,6,7,8,9,...,155,156,157,158,159,160,161,162,163,165
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104394,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
104395,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
104396,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
104397,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
data = data.drop(['char_jobcode'], axis=1)
data.shape

(104399, 38)

In [18]:
data_hour = pd.to_datetime(data['logging_timestamp']).dt.strftime('%H').astype(int)
data_hour = data_hour.replace(0, 24)
data['char_onehot'] = level_onehot
data.shape

ValueError: Wrong number of items passed 165, placement implies 1

In [20]:
data

,newID,char_level,logging_timestamp,charStatA,charStatB,charStatC,charStatD,charStatE,charStatF,charStatG,...,actionSessionAmountG,actionSessionAmountH,actionSessionAmountI,actionSessionAmountJ,actionSessionAmountK,actionSessionAmountL,actionSessionAmountM,actionSessionAmountN,tradeSessionAmountD,blocked
0,0,10,2017-6-26 8:59:4.224398,0,0,0,0,0,0,0,...,0,0,0,0,0,2,0,0,0,0
1,0,31,2017-6-26 9:29:6.39562,0,0,0,0,2,0,0,...,0,0,0,0,2,146,0,106,0,0
2,0,40,2017-6-26 9:59:13.175257,0,0,0,0,2,0,0,...,0,0,0,0,2,229,0,172,0,0
3,0,61,2017-6-26 10:29:13.525274,0,0,0,0,6,0,0,...,1,1,2,0,2,583,0,615,0,0
4,0,61,2017-6-27 7:51:56.193386,0,0,0,0,6,0,0,...,0,0,1,0,17,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104394,14915,47,2017-6-19 16:47:28.622771,1,0,4563,0,4575,0,1,...,1,0,4,0,0,89,0,59,0,0
104395,14915,16,2017-6-19 12:17:5.647646,0,0,4563,0,4563,0,0,...,0,0,0,0,0,45,0,44,0,0
104396,14915,23,2017-6-19 12:47:8.165005,0,0,4563,0,4565,0,0,...,0,0,0,0,0,195,0,153,0,0
104397,14915,27,2017-6-19 13:17:9.600144,0,0,4563,0,4565,0,0,...,0,0,1,0,0,193,0,152,0,0
